In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import ijson
import urllib.parse
import math
import sqlite3

# Commit the changes and close the connection
conn = sqlite3.connect('cards_database.db')



In [2]:
def process_data(soup):
    # Initialize a list to hold card data
    cards_data = []

    # Find all card rows
    for row in soup.find_all('div', class_=('row','amtAndPrice')):
        if not row:
            continue
        card_info = {}
        
        try:    # Extract card name
            name_tag = row.find('a', class_='item-name-bold')
            if name_tag:
                card_info['Name'] = name_tag.text.strip()
            
            # Extract brand
            brand_tag = row.find(text='Brand:').find_next('p')
            if brand_tag:
                card_info['Brand'] = brand_tag.text.strip()
            
            # Extract set
            set_tag = row.find(text='Set:').find_next('p')
            if set_tag:
                card_info['Set'] = set_tag.text.strip()
            

            set_link_tag = row.find('p', class_='item-text', text='Set:').find_next('a')
            if set_link_tag:
                card_info['Set'] = set_link_tag.text.strip()
                # Extract the product ID from the link
                product_id = set_link_tag['href'].split('item_set=')[-1]
                card_info['Product ID'] = product_id
            # Extract language
            language_tag = row.find(text='Language:').find_next('p')
            if language_tag:
                card_info['Language'] = language_tag.text.strip()
            
            # Extract finish
            finish_tag = row.find(text='Finish:').find_next('p')
            if finish_tag:
                card_info['Finish'] = finish_tag.text.strip()
            
            # Extract rarity
            rarity_tag = row.find(text='Rarity:').find_next('p')
            if rarity_tag:
                card_info['Rarity'] = rarity_tag.text.strip()
            
            # Extract status
            status_tag = row.find(text='Status:').find_next('p')
            if status_tag:
                card_info['Status'] = status_tag.text.strip()
            
            stock_details = {}
            stock_qty_tag = row.find('span', class_='styleQty')
            if stock_qty_tag:
                card_info['Stock Quantity'] = stock_qty_tag.text.strip()
            else:
                card_info['Stock Quantity'] = 0

            
            stock_avail_tag = row.find('span', class_='styleQtyAvailText')
            if stock_avail_tag:
                card_info['Stock Availability'] = stock_avail_tag.text.strip()
            else:
                card_info['Stock Availability'] = 0

            stock_price_tag = row.find('span', class_='stylePrice')
            if stock_price_tag:
                card_info['Stock Price'] = stock_price_tag.text.strip()
            else:
                card_info['Stock Price'] = 0
            #  the card info to the list
            cards_data.append(card_info)
        except Exception as e:
            continue
    # Create a DataFrame
    df = pd.DataFrame(cards_data)
    return df.fillna(0) 

In [3]:
def get_max_page(url):
    response = requests.get(url)
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the content of the page
        soup = BeautifulSoup(response.content, 'html.parser')
        total_records = int(soup.find('span', id='total-record-number-top').text.strip())
        return math.ceil(total_records/60) + 1
    else:
        return None

In [4]:
def web_scraper(parsed_txt, maxpages):
    combined_df = pd.DataFrame()
    for i in range(1, maxpages):
        response = requests.get(f'https://www.classroomtgc.com/Product/Index2?type=sell&item_name={parsed_txt}&item_brand=&before_price=0&after_price=0&page_number={i}&record_number=60&sort_by=')
        
        if not response.status_code == 200:
            continue

        soup = BeautifulSoup(response.content, 'html.parser')
        df = process_data(soup)
        combined_df = pd.concat([combined_df, df])
    # Extract and return the title of the page
    return combined_df
    # else:
    #     return f"Error: Unable to retrieve page, status code {response.status_code}"


In [6]:
def insert_db(cursor, data):
    for index, row in data.iterrows():
        try:
            cursor.execute('''
                INSERT INTO cards (name, brand, set_name, language, finish, rarity, status, stock_quantity, stock_availability, stock_price)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?) ON CONFLICT DO NOTHING;
            ''', (row['Name'], row['Brand'], row['Set'], row['Language'], row['Finish'], row['Rarity'], row['Status'], row['Stock Quantity'], row['Stock Availability'], row['Stock Price']))
            cursor.commit()
        except:
            continue


In [7]:

cursor = conn.cursor()

with open('all-cards-20241118101926.json', 'r') as f:
    # Use ijson's basic iterator to parse the file incrementally
    objects = ijson.items(f, 'item')  # Replace 'item' with the correct key if needed
    for obj in objects:
        # Process each object one at a time
        print(obj['name'])
        existing_entry = cursor.execute('SELECT * FROM cards WHERE name = ?', (obj['name'],)).fetchone()
        
        if existing_entry:
            continue
        else:
            print('not existing_entry')

        parsed_txt = urllib.parse.quote(obj['name'], safe='')
        parsed_txt = parsed_txt.replace('%20','+')
        print(parsed_txt)
        
        maxpages = get_max_page(f'https://www.classroomtgc.com/Product/Index2?type=sell&item_name={parsed_txt}&item_brand=&before_price=0&after_price=0&page_number=1&record_number=60&sort_by=')
        print(maxpages)
        if not maxpages:
            continue

        data = web_scraper(parsed_txt, maxpages)

        # display(data)
        insert_db(cursor, data)
        conn.commit()



Forest
not existing_entry
Forest
24


C:\Users\Owner\AppData\Local\Temp\ipykernel_15088\1759385609.py:17: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  brand_tag = row.find(text='Brand:').find_next('p')
C:\Users\Owner\AppData\Local\Temp\ipykernel_15088\1759385609.py:22: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  set_tag = row.find(text='Set:').find_next('p')
C:\Users\Owner\AppData\Local\Temp\ipykernel_15088\1759385609.py:27: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  set_link_tag = row.find('p', class_='item-text', text='Set:').find_next('a')
C:\Users\Owner\AppData\Local\Temp\ipykernel_15088\1759385609.py:34: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  language_tag = row.find(text='Language:').find_next('p')
C:\Users\Owner\AppData\Local\Temp\ipykernel_15088\1759385609.py:39: DeprecationWarning: 

Fury Sliver
not existing_entry
Fury+Sliver
2


C:\Users\Owner\AppData\Local\Temp\ipykernel_15088\1759385609.py:17: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  brand_tag = row.find(text='Brand:').find_next('p')
C:\Users\Owner\AppData\Local\Temp\ipykernel_15088\1759385609.py:22: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  set_tag = row.find(text='Set:').find_next('p')
C:\Users\Owner\AppData\Local\Temp\ipykernel_15088\1759385609.py:27: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  set_link_tag = row.find('p', class_='item-text', text='Set:').find_next('a')
C:\Users\Owner\AppData\Local\Temp\ipykernel_15088\1759385609.py:34: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  language_tag = row.find(text='Language:').find_next('p')
C:\Users\Owner\AppData\Local\Temp\ipykernel_15088\1759385609.py:39: DeprecationWarning: 

Kor Outfitter
not existing_entry
Kor+Outfitter
2


...

Glacian, Powerstone Engineer
not existing_entry
Glacian%2C+Powerstone+Engineer
2


C:\Users\Owner\AppData\Local\Temp\ipykernel_15088\1759385609.py:17: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  brand_tag = row.find(text='Brand:').find_next('p')
C:\Users\Owner\AppData\Local\Temp\ipykernel_15088\1759385609.py:22: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  set_tag = row.find(text='Set:').find_next('p')
C:\Users\Owner\AppData\Local\Temp\ipykernel_15088\1759385609.py:27: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  set_link_tag = row.find('p', class_='item-text', text='Set:').find_next('a')
C:\Users\Owner\AppData\Local\Temp\ipykernel_15088\1759385609.py:34: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  language_tag = row.find(text='Language:').find_next('p')
C:\Users\Owner\AppData\Local\Temp\ipykernel_15088\1759385609.py:39: DeprecationWarning: 

Swamp
not existing_entry
Swamp
22


C:\Users\Owner\AppData\Local\Temp\ipykernel_15088\1759385609.py:17: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  brand_tag = row.find(text='Brand:').find_next('p')
C:\Users\Owner\AppData\Local\Temp\ipykernel_15088\1759385609.py:22: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  set_tag = row.find(text='Set:').find_next('p')
C:\Users\Owner\AppData\Local\Temp\ipykernel_15088\1759385609.py:27: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  set_link_tag = row.find('p', class_='item-text', text='Set:').find_next('a')
C:\Users\Owner\AppData\Local\Temp\ipykernel_15088\1759385609.py:34: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  language_tag = row.find(text='Language:').find_next('p')
C:\Users\Owner\AppData\Local\Temp\ipykernel_15088\1759385609.py:39: DeprecationWarning: 

KeyboardInterrupt: 

In [38]:
data

,Name,Brand,Set,Product ID,Language,Finish,Rarity,Status,Stock Quantity,Stock Availability,Stock Price
0,Admiral Beckett Brass,Magic: the Gathering,The List,2540,English,nonfoil,Mythic,Active,8,available,MYR 5
1,Admiral Beckett Brass,Magic: the Gathering,Ixalan,3030,English,nonfoil,Mythic,Active,8,available,MYR 5
2,Admiral Beckett Brass,Magic: the Gathering,Ixalan,3030,English,foil,Mythic,Active,0,0,MYR 31
3,Admiral Beckett Brass (Prerelease Datestamped),Magic: the Gathering,Ixalan Promos,2944,English,foil,Mythic,Active,0,0,MYR 46
4,Admiral Beckett Brass,Magic: the Gathering,The Lost Caverns of Ixalan Commander,100042,English,nonfoil,Mythic,Active,0,0,MYR 5
5,Admiral Beckett Brass,Magic: the Gathering,The Lost Caverns of Ixalan Commander,100042,English,foil,Mythic,Active,0,0,MYR 6
6,Admiral Beckett Brass (Showcase),Magic: the Gathering,Secret Lair Drop,2443,English,foil,Mythic,Active,0,0,MYR 6
7,Admiral Beckett Brass (Showcase),Magic: the Gathering,Secret Lair Drop,2443,English,nonfoil,Mythic,Active,0,0,MYR 28
8,Admiral Beckett Brass (Showcase),Magic: the Gathering,Secret Lair Drop,2443,English,foil,Mythic,Active,0,0,MYR 40
9,Admiral Beckett Brass (Showcase),Magic: the Gathering,Secret Lair Drop,2443,English,foil,Mythic,Active,0,0,MYR 40
